# **Rss Parsing And Json Conversion**


---
Στο παρακάτω κομμάτι γίνεται μετατροπή των RSS στοιχείων σε Json format για πιο εύκολη επεξεργασία τους.


In [ ]:
import urllib.request
import json


url='https://api.rss2json.com/v1/api.json?rss_url=https%3A%2F%2Fec.europa.eu%2Fcommission%2Fpresscorner%2Fapi%2Frss%3Flanguage%3Den%26documenttype%3D852%26commissioner%3D%26pagesize%3D100&api_key=cne8r2iixz2orfwbisnqbfcmeg3weyltcqqmczto&count=100'

response = urllib.request.urlopen(url)

data = json.loads(response.read())

for i in range(len(data['items'])):
    del data['items'][i]['author']
    del data['items'][i]['enclosure']
    del data['items'][i]['content']
    del data['items'][i]['guid']
    del data['items'][i]['thumbnail']
final_data = json.dumps(data['items'], indent=4)

with open('data.json', 'w') as outfile:
    outfile.write(final_data)

In [ ]:
from networkx.algorithms.centrality import load
with open("data.json", "r+") as test:
  newtest=json.load(test)

Τα block 3 - 6 είναι η εγκατάση των library που χρειαζομαστε για το haystack και το elasticsearch και η ενεργοποίηση του elasticsearch 

In [ ]:
!pip3 install farm-haystack

!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]
!pip install python-magic


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 524 kB 33.7 MB/s 
     |████████████████████████████████| 43 kB 2.6 MB/s 
     |████████████████████████████████| 374 kB 54.3 MB/s 
     |████████████████████████████████| 79 kB 8.6 MB/s 
     |████████████████████████████████| 50 kB 7.9 MB/s 
     |████████████████████████████████| 5.6 MB 4.0 MB/s 
     |████████████████████████████████| 11.1 MB 45.1 MB/s 
     |████████████████████████████████| 3.3 MB 47.2 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:41tcmalloc: large alloc 1147494400 bytes == 0x3ab94000 @  0x7f496d3d9615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |███████████████████████████

In [ ]:
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.nodes import EmbeddingRetriever
import pandas as pd
import requests

INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
ERROR - root -  Failed to import 'magic' (from 'python-magic' and 'python-magic-bin' on Windows). FileTypeClassifier will not perform mimetype detection on extensionless files. Please make sure the necessary OS libraries are installed if you need this functionality.


In [ ]:
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)
# wait until ES has started
! sleep 30

In [ ]:
from haystack.document_stores import ElasticsearchDocumentStore

document_store = ElasticsearchDocumentStore(
    host="localhost",
    username="",
    password="",
    index="document",
    embedding_field="question_emb",
    embedding_dim=384,
    excluded_meta_data=["question_emb"],
)

retriever = EmbeddingRetriever(
    document_store=document_store, embedding_model="sentence-transformers/all-MiniLM-L6-v2", use_gpu=True
)

INFO - haystack.telemetry -  Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://haystack.deepset.ai/guides/telemetry
INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.nodes.retriever.dense -  Init retriever using embeddings of model sentence-transformers/all-MiniLM-L6-v2
WARNING - haystack.nodes.retriever.dense -  You seem to be using a Sentence Transformer embedding model but 'model_format' is set to 'farm'. You may need to set 'model_format='sentence_transformers' to ensure correct loading of model.
INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number o

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/86.7M [00:00<?, ?B/s]

INFO - haystack.modeling.model.language_model -  Loaded sentence-transformers/all-MiniLM-L6-v2


Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

INFO - haystack.modeling.data_handler.processor -  Initialized processor without tasks. Supply `metric` and `label_list` to the constructor for using the default task or add a custom task later via processor.add_task()
INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
WARNING - haystack.nodes.retriever._embedding_encoder -  You seem to be using a Sentence Transformer with the dot_product function. We recommend using cosine instead. This can be set when initializing the DocumentStore


In [ ]:
with open("data.json") as search:
  search_data = json.load(search)

In [ ]:
print(search_data)

Εγκαθηστω το selenium για να παρω τα κομματια κειμενου που θελουμε για τις ερωτησεις και τις απαντησεις απο τα 100 site. 

In [ ]:
print(search_data[4]["link"])

https://ec.europa.eu/commission/presscorner/detail/en/QANDA_20_2348


In [ ]:
!pip install selenium
!apt-get update && upgrade
!apt install chromium-chromedriver
from selenium import webdriver
import time
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)


for f in range(100):
  driver.get(search_data[f]['link'])
  time.sleep(1)
  with open(f'searches{f}.txt', 'w') as outfile:
      text = driver.find_element_by_id("inline-nav-1").get_attribute("innerText")
      outfile.write(text)
driver.quit()

Στην συνεχεια γινεται επεξεργασια των αρχειων που φτιαξαμε με τα στοιχεια του καθε site και τα τοποθετω σε ενα καινουριο αρχειο Json.

In [ ]:
from numpy import result_type

keys=range(100)

results_dict={}
py={**results_dict,**{"Question":[],"Answer":[]}}
py={**results_dict,**{"Question":[],"Answer":[]}}

for i in keys:
  out=" "

  with open(f'searches{i}.txt',"r+") as f:
      lines = f.readlines()
      # το συγκεκριμενο flag δημιουργειται γιατι σε μερικα κειμενα δεν υπηρχε στο κειμενο ερωτηση 
      # και δεν μπορουσα να βρω καποιο τροπο να παιρνω τα κομματια ερωτησης και απαντησης που να ειναι αποδοτικος
      flag = False
      for line in lines:
        if line.find("?") != -1 :
          flag = True
        
      #Οι έλεγχοι ειναι απλοί. Αν η γραμμή τελειώνει με το ?\n τοτε θεωρειται ερωτηση.
      # Αν οχι τοτε ειναι απαντηση (Απο την απαντηση εξαιρουνται ολες οι κενες γραμμες .isspace():continue
      # και απο τα κειμενα δεν μας ενδιαφερει και το κομματι με τις πληροφοριες
      # οποτε απο οταν βρισκει την γραμμη for more information η λουπα για την επαιξεργασια του κειμενου σπαει και παει στο επομενο

      if flag:
        for line in lines:
          if line.endswith("?\n"):
            if not out.isspace():
              py["Question"].append(line)
              py["Answer"].append(out)
            out=" "
          elif line.startswith("For More Information"):
            break
          elif line.isspace():
            continue
          else:
            out+=line
            # results_dict["Question"].append(line)
      else:
        break
  final_results = json.dumps(py, indent=4) 

  with open(f'final_results.json',"w+") as new_file:
        new_file.write(final_results)

Στα τελευταία Block γίνεται το search για την ερώτηση που θα κάνουμε στο block 36.

In [ ]:
from haystack.document_stores import ElasticsearchDocumentStore

document_index = "document"
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index=document_index)


from haystack.utils import fetch_archive_from_http

# Download
doc_dir = "/content/"

# Get dataframe with columns "question", "answer" and some custom metadata
df = pd.read_json(f"{doc_dir}/final_results.json")
# Minimal cleaning
df.fillna(value="", inplace=True)
df["Question"] = df["Question"].apply(lambda x: x.strip())

# Get embeddings for our questions from the FAQs
questions = list(df["Question"].values)
df["question_emb"] = retriever.embed_queries(texts=questions)
df = df.rename(columns={"Question": "content"})

# Convert Dataframe to list of dicts and index them in our DocumentStore
docs_to_index = df.to_dict(orient="records")
document_store.write_documents(docs_to_index)

Inferencing Samples: 100%|██████████| 3/3 [00:00<00:00,  3.26 Batches/s]


In [ ]:
from haystack.pipelines import FAQPipeline

pipe = FAQPipeline(retriever=retriever)

In [ ]:
from haystack.utils import print_answers

user_input=input("Ask your question:")

prediction = pipe.run(query=user_input, params={"Retriever": {"top_k": 5}})
for i in range(5):
  print("\nTop Answer", (i+1) ,"of 5", prediction["answers"][i].meta.get("Answer"))